In [1]:
%load_ext autoreload
%autoreload 2
    
import fiona
from pathlib import Path
import metapack as mp
import geopandas as gpd
import pandas as pd
import numpy as np
from auto_tqdm import tqdm 
import appnope

doc = mp.jupyter.open_source_package()
doc.set_sys_path()
import  pylib 

ea_epsg = 2163 #US Equal Area projection

import logging
logging.basicConfig()

from pylib import lines_logger, points_logger
lines_logger.setLevel(logging.DEBUG)
points_logger.setLevel(logging.DEBUG)

pkg_root = Path(doc.path).parent
pkg = mp.open_package(pkg_root)
cache = pylib.open_cache(pkg)
pkg


# Open Street Map Extract for Demographic Search
`civicknowledge.com-osm-demosearch-1.2.1` Last Update: 2021-02-15T17:38:34

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [point_tags](data/point_tags.csv)**. Points converted to counts of tags per geohash
* ** [residential_roads](data/residential_roads.csv)**. Residential roads per 4 digit geohash
* ** [nonres_roads](data/nonres_roads.csv)**. Non residential roads per 4 digit geohash
* ** [block_geo](data/block_geo.csv)**. Geographic shapes for blocs in block_osm
* ** [block_osm](data/block_osm.csv)**. Points from geohash_tags allocated to 2020 census blocks
* ** [utm_map](data/utm_map.csv)**. Map from census blocks to UTM zones
* ** [cbsa_map](data/cbsa_map.csv)**. Map from census blocks to CBSA Metros
* ** [bus_densities](notebooks/PointDensity.ipynb#bus_densities)**. High-density business goehashes

## References
<ul><li> <strong>us_geohashes</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_geohashes</em>. All 4 digit geohases in the continential US</li><li> <strong><a href="https://download.geofabrik.de/north-america-latest.osm.pbf">north-america-latest</a></strong>. OSM North America extract</li><li> <strong>utm_grid</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-mgrs.csv#utm_grid</em>. </li><li> <strong>points</string>, <em>data/csv/points.csv</em>. Points from the OSM file</li><li> <strong>lines</string>, <em>data/csv/lines.csv</em>. Lines from the OSM file</li><li> <strong>multipolygons</string>, <em>data/csv/multipolygons.csv</em>. Polygons from the OSM file</li><li> <strong>multilinestrings</string>, <em>data/csv/multilinestrings.csv</em>. Lines from the OSM file</li><li> <strong>other_relations</string>, <em>data/csv/other_relations.csv</em>. Other geo data from the OSM file</li><li> <strong>block_templ</string>, <em>censusgeo://2020/5/{st}/block</em>. Block url template</li><li> <strong>cbsa</string>, <em>censusgeo://2020/5/US/cbsa</em>. Metro areas</li><ul>

In [2]:
%time pt  = pkg.resource('point_tags').geoframe()

CPU times: user 3min 56s, sys: 3.46 s, total: 3min 59s
Wall time: 4min 1s


In [3]:
from demosearch import FileCache
from demosearch.search import ftsearch
import utm
from collections import defaultdict

fc = FileCache('/Volumes/SSD_Extern/radius/')
def ftbb(s):
    bb = ftsearch(fc, s)[0].bb
    return (slice(bb[0],bb[2]), slice(bb[1],bb[3]))

ftbb('San Diego')


(slice(-117.611081, -116.08094, None),
 slice(32.528832, 33.505024999999996, None))

In [4]:
r=ftsearch(fc, 'San Diego')[0]
sd = pt[pt.geoid==r.geoid]



In [5]:
sd.to_csv('business_points.csv')

In [51]:

from sklearn.cluster import KMeans
import numpy as np

sdu = sd.to_crs(32611)
km = pd.DataFrame({
        'gh': sdu.geohash,
        'x': sdu.geometry.x,
        'y': sdu.geometry.y
})

X = km[['x','y']]
kmeans = KMeans(n_clusters=2000, random_state=0).fit(X)

km['cluster'] = kmeans.predict(X)
km['geometry'] = gpd.points_from_xy(km.x, km.y, crs=sdu.crs)
km.to_csv('kmeans_clusters.csv')

In [48]:
t = km.cluster.value_counts()
lg_clusters = t[t>20].index

In [49]:
km[km.cluster.isin(lg_clusters)].to_csv('kmeans_clusters.csv')

In [50]:
lg_clusters

Int64Index([ 65,  74,  15, 532, 180, 135,  16, 828, 978, 719,
            ...
            392, 159, 347, 673, 713, 720, 320, 815, 428, 569],
           dtype='int64', length=194)